## Computer

This notebook demonstrates how to use Computer to operate sandbox VMs programmatically, either using C/ua Cloud Containers or local Lume VMs on Apple Silicon macOS systems.

### Installation

In [ ]:
!pip uninstall -y cua-computer
!pip install "cua-computer[all]"

If locally installed, use this instead:

In [ ]:
import os

os.chdir('../libs/computer')
!poetry install
!poetry build

!pip uninstall cua-computer -y
!pip install ./dist/cua_computer-0.1.0-py3-none-any.whl --force-reinstall

## Option 1: C/ua Cloud Containers

C/ua Cloud Containers provide remote VMs that can be accessed from any system without local setup.

### Prerequisites for Cloud Containers

To use C/ua Cloud Containers, you need to:
1. Sign up at https://trycua.com
2. Create a Cloud Container
3. Generate an API Key

Once you have these, you can connect to your cloud container using its name.

In [ ]:
# Get API key and container name from environment or prompt user
import os

cua_api_key = os.getenv("CUA_API_KEY") or \
    input("Enter your C/ua API Key: ")
container_name = os.getenv("CONTAINER_NAME") or \
    input("Enter your Cloud Container name: ")

Choose the OS type for your container (linux or macos)

In [ ]:
os_type = input("Enter the OS type of your container (linux/macos) [default: linux]: ").lower() or "linux"

### Connect to your Cloud Container

In [ ]:
from computer import Computer, VMProviderType

Connect to your existing C/ua Cloud Container

In [ ]:
computer = Computer(
    os_type=os_type,  # Must match the OS type of your cloud container
    api_key=os.getenv("CUA_API_KEY"),
    name=os.getenv("CONTAINER_NAME"),
    provider_type=VMProviderType.CLOUD,
)

Take a screenshot

In [ ]:
screenshot = await computer.interface.screenshot()

with open("screenshot.png", "wb") as f:
    f.write(screenshot)

## Option 2: Local VMs (Lume daemon)

For Apple Silicon Macs, you can run VMs locally using the Lume daemon.

### Lume daemon setup

Refer to [../libs/lume/README.md](../libs/lume/README.md) for more details on the lume cli.

For installing the standalone lume binary, run the following command from a terminal:

```bash
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/trycua/cua/main/libs/lume/scripts/install.sh)"
```

Pull the latest pre-built macos-sequoia-cua image. This image, based on macOS Sequoia, contains all dependencies needed to be controlled from the cua-computer interface.

In [ ]:
!lume pull macos-sequoia-cua:latest

VMs are stored in `~/.lume`, and locally cached images are stored in `~/.lume/cache`.

You can always see the list of downloaded VM images with:

In [5]:
!lume ls

name                              os      cpu     memory  disk            display     status          storage         shared_dirs                                           ip              vnc                                               
macos-sequoia-cua_latest          macOS   4       8.00G   25.9GB/80.0GB   1024x768    stopped         default         -                                                     -               -                                                 


## Testing the sandbox

Once downloaded, you can run the sandbox to test everything is working:

```bash
lume run macos-sequoia-cua:latest
```

You can add additional software and tools to the sandbox - these changes will be saved in the VM disk.

### Initialize a Local Computer instance

Computer allows you to create and control a virtual sandbox instances from your host on Apple Silicon. Here's a basic example:

In [23]:
from computer import Computer, VMProviderType

You can either use the async context manager or initialize the Computer instance directly.

With the async context manager:

In [ ]:
async with Computer(
    # name="my_vm", # optional, in case you want to use any other VM created using lume
    display="1024x768",
    memory="8GB",
    cpu="4",
    os_type="macos",
    provider_type=VMProviderType.LUME,
) as computer:
    pass
    # ... do something with the computer interface

Direct initialization:

In [ ]:
computer = Computer(
    display="1024x768",
    memory="8GB",
    cpu="4",
    os_type="macos"
)

await computer.run()

## Computer Interface

Both cloud and local computers provide the same interface for interaction.

In order to execute commands targeting the sandbox, the Computer interface communicates through websockets to a Fast API server running on the sandbox.

### Cursor

In [8]:
# Move and click
await computer.interface.move_cursor(100, 100)
await computer.interface.left_click()
await computer.interface.right_click(300, 300)
await computer.interface.double_click(400, 400)

# Drag operations
await computer.interface.drag_to(500, 500, duration=1.0)

# Get cursor position
cursor_pos = await computer.interface.get_cursor_position()

### Keyboard

In [9]:
# Type text
await computer.interface.type_text("Hello, World!")

# Press individual keys
await computer.interface.press_key("enter")
await computer.interface.press_key("escape")

# Use hotkeys
await computer.interface.hotkey("command", "c")  # Command+C on macOS

### Screen

In [10]:
# Get screen dimensions
screen_size = await computer.interface.get_screen_size()

# Take basic screenshot
screenshot = await computer.interface.screenshot()
with open("screenshot.png", "wb") as f:
    f.write(screenshot)

### Clipboard

In [ ]:
# Set clipboard content
await computer.interface.set_clipboard("Text to copy")

# Get clipboard content
clipboard_content = await computer.interface.copy_to_clipboard()

### File System Operations

In [ ]:
# Check file/directory existence
file_exists = await computer.interface.file_exists("/path/to/file.txt")
dir_exists = await computer.interface.directory_exists("/path/to/directory")

# Run shell commands
stdout, stderr = await computer.interface.run_command("ls -la")

### Accessibility

In [ ]:
# Get accessibility tree
accessibility_tree = await computer.interface.get_accessibility_tree()
print(accessibility_tree)

### Sharing a directory with the sandbox

You can share a directory with the sandbox by passing a list of absolute paths to the `shared_directories` argument when initializing the Computer instance.

In [ ]:
computer = Computer(
    display="1024x768",
    memory="4GB",
    cpu="2",
    os_type="macos",
    shared_directories=["/absolute/path/to/directory"]
)

### Using Host Computer

For local development and testing purposes, you can run the Computer API server on the same host machine and target it instead:

In [ ]:
computer = Computer(
    display="1024x768",
    memory="4GB",
    cpu="2",
    os_type="macos",
    use_host_computer_server=True
)

### Examples

In [3]:
from computer.computer import Computer

In [ ]:
async with Computer(
    display="1024x768",
    memory="4GB",
    cpu="2",
    os_type="macos"
) as computer:
    await computer.run()
    res = await computer.interface.run_command("ls -a")

    # Get screen dimensions
    screen_size = await computer.interface.get_screen_size()

    # Move and click
    await computer.interface.move_cursor(100, 100)
    await computer.interface.left_click()
    await computer.interface.right_click(300, 300)
    await computer.interface.double_click(400, 400)

    # Drag operations
    await computer.interface.drag_to(500, 500, duration=1.0)

    # Get cursor position
    cursor_pos = await computer.interface.get_cursor_position()

    # Your automation code here
    await computer.stop()